In [2]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath('..'))
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, VBox, HBox
import numpy as np
from torch.utils.data import DataLoader, random_split
import random
from src.dataset import ProjectDatasets
from src.models.UNet import UNet
from src.utils.helpers import visualize_batch_with_colorbar
from torchvision.transforms import Compose, ToTensor, Normalize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
checkpoints_path = "/net/travail/rramesh/AdvanceimageProcessing/Semantic-Segmentation-for-Autonomous-Driving/experiments/checkpoint_epoch_10.pth"
checkpoints_path = "/net/cremi/sasifchaudhr/espaces/travail/Semantic-Segmentation-for-Autonomous-Driving/experiments/experiment_20241220-131602/checkpoints/checkpoint_epoch_10.pth"

transform = Compose([ToTensor(), Normalize(mean=[0.5], std=[0.5])])
dataset = ProjectDatasets(root_path="/net/ens/am4ip/datasets/project-dataset", transform=transform)
sample = dataset[0]  # Should return 4 values
print(len(sample))  # Should print 4

model = UNet(in_channels=3, out_channels=30)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Pre-warm CUDA
if torch.cuda.is_available():
    _ = torch.randn(1).to(device)

checkpoint = torch.load(checkpoints_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])


4


/tmp/cache-sasifchaudhr/ipykernel_2472603/710902063.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoints_path, map_location=device)


<All keys matched successfully>

In [4]:
# Split dataset
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
generator = torch.Generator().manual_seed(seed)
_, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

In [5]:
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [6]:
# for batch_idx, (inputs, targets, _) in enumerate(val_loader):
#     with torch.no_grad():
#         inputs, targets = inputs.to(device), targets.to(device)
#         predictions = model(inputs)
#         predictions = torch.argmax(predictions, dim=1)  # Assuming output is logits
        
#         print(f"The shape of the prediction is {predictions.shape}")  
#        # Check intensity range of predictions
#         max_intensity = torch.max(predictions).item()
#         min_intensity = torch.min(predictions).item()
#         print(f"The maximum intensity of the prediction is {max_intensity} and the minimum is {min_intensity}")
        
#     visualize_batch_with_colorbar(inputs, predictions, targets, batch_idx, num_samples=3)
#     break# Visualize only the first batch

# Mapping RGB pixel in prediction

In [7]:
# Reverse the color_to_class mapping to get class_to_color mapping
color_to_class = {
    (0, 0, 0): 0,            # Unlabeled
    (111, 74, 0): 1,         # Dynamic
    (81, 0, 81): 2,          # Ground
    (128, 64, 128): 3,       # Road
    (244, 35, 232): 4,       # Sidewalk
    (250, 170, 160): 5,      # Parking
    (230, 150, 140): 6,      # Rail track
    (70, 70, 70): 7,         # Building
    (102, 102, 156): 8,      # Wall
    (190, 153, 153): 9,      # Fence
    (180, 165, 180): 10,     # Guard rail
    (150, 100, 100): 11,     # Bridge
    (150, 120, 90): 12,      # Tunnel
    (153, 153, 153): 13,     # Pole
    (250, 170, 30): 14,      # Traffic light
    (220, 220, 0): 15,       # Traffic sign
    (107, 142, 35): 16,      # Vegetation
    (152, 251, 152): 17,     # Terrain
    (70, 130, 180): 18,      # Sky
    (220, 20, 60): 19,       # Person
    (255, 0, 0): 20,         # Rider
    (0, 0, 142): 21,         # Car
    (0, 0, 70): 22,          # Truck
    (0, 60, 100): 23,        # Bus
    (0, 0, 90): 24,          # Caravan
    (0, 0, 110): 25,         # Trailer
    (0, 80, 100): 26,        # Train
    (0, 0, 230): 27,         # Motorcycle
    (119, 11, 32): 29        # Bicycle
}

class_to_color = {v: k for k, v in color_to_class.items()}

def map_classes_to_colors(predictions, class_to_color):
    """
    Map class IDs to RGB colors for a segmentation map.

    Args:
        predictions (torch.Tensor): Tensor of shape (H, W) containing class IDs.
        class_to_color (dict): Mapping from class IDs to RGB colors.

    Returns:
        np.ndarray: RGB image of shape (H, W, 3).
    """
    height, width = predictions.shape
    rgb_image = np.zeros((height, width, 3), dtype=np.uint8)
    
    for class_id, color in class_to_color.items():
        rgb_image[predictions == class_id] = color  # Map each class to its color
    
    return rgb_image

In [8]:
# Example: Visualizing predictions
for batch_idx, (inputs, targets, _) in enumerate(val_loader):
    with torch.no_grad():
        inputs, targets = inputs.to(device), targets.to(device)
        predictions = model(inputs)
        predictions = torch.argmax(predictions, dim=1)  # Shape: [N, H, W]

        print(f"the shape of the targets is {targets.shape}")
        print(f"the shape of the inputs is {inputs.shape}")
        
        # Convert all predictions in the batch to RGB format
        rgb_predictions = np.stack([
            map_classes_to_colors(predictions[i].cpu().numpy(), class_to_color) for i in range(predictions.shape[0])
        ])  # Shape: [N, H, W, 3]
        

        rgb_targets = np.stack([
            map_classes_to_colors(targets[i].cpu().numpy(), class_to_color) for i in range(targets.shape[0])
        ])


    # Visualize the batch with RGB predictions
    visualize_batch_with_colorbar(inputs, rgb_predictions, rgb_targets, batch_idx, num_samples=8, rgb_pred=True)
    break



ValueError: too many values to unpack (expected 3)

In [ ]:

class VideoPlayer:
    def __init__(self, inputs, predictions, targets, sources, image_names, rgb_pred=False, interval=0.5):
        # Same initialization as before
        self.inputs = inputs.cpu().numpy()
        self.predictions = predictions
        self.targets = targets if isinstance(targets, np.ndarray) else targets.cpu().numpy()
        self.sources = sources
        self.image_names = image_names
        self.rgb_pred = rgb_pred
        self.num_samples = inputs.shape[0]
        self.interval = interval  # Playback speed in seconds

        # State variables
        self.current_index = 0
        self.playing = False

        # Widgets
        self.slider = widgets.IntSlider(value=0, min=0, max=self.num_samples - 1, step=1, description="Frame:")
        self.play_button = widgets.ToggleButton(value=False, description="Play", button_style="success")
        self.next_button = widgets.Button(description="Next")
        self.prev_button = widgets.Button(description="Previous")
        self.output = widgets.Output()

        # Event bindings
        self.play_button.observe(self.toggle_play, names="value")
        self.slider.observe(self.update_frame, names="value")
        self.next_button.on_click(self.next_frame)
        self.prev_button.on_click(self.previous_frame)

        # Layout
        self.controls = widgets.HBox([self.prev_button, self.play_button, self.next_button, self.slider])
        display(self.controls, self.output)

    def display_frame(self, index):
        """
        Display the frame corresponding to the given index.
        """
        with self.output:
            # Ensure the output area is cleared before redrawing
            clear_output(wait=True)

            # Create a new figure for each frame
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Normalize inputs, predictions, and targets
            normalized_input = (self.inputs[index].transpose(1, 2, 0) + 1) / 2  # Scale inputs to [0, 1]
            normalized_predictions = (
                self.predictions[index] / 255.0 if self.predictions[index].max() > 1 else self.predictions[index]
            )
            normalized_targets = (
                self.targets[index] / 255.0 if self.targets[index].max() > 1 else self.targets[index]
            )

            # Input image
            ax1 = axes[0]
            ax1.imshow(normalized_input, vmin=0, vmax=1)
            ax1.set_title(f"Input Image (Frame {index})", fontsize=12)
            ax1.axis("off")

            # Prediction
            ax2 = axes[1]
            ax2.imshow(normalized_predictions, vmin=0, vmax=1)
            ax2.set_title(f"Prediction (Frame {index})", fontsize=12)
            ax2.axis("off")

            # Ground truth
            ax3 = axes[2]
            ax3.imshow(normalized_targets, vmin=0, vmax=1)
            ax3.set_title(f"Ground Truth (Frame {index})", fontsize=12)
            ax3.axis("off")

            # Add source and image name
            fig.suptitle(
                f"Source: {self.sources[index]} | Image Name: {self.image_names[index]}",
                fontsize=14,
                y=0.98,
            )

            # Display the figure
            plt.tight_layout()
            plt.show()


    def toggle_play(self, change):
        """
        Toggle play/pause functionality.
        """
        self.playing = change["new"]
        if self.playing:
            self.play_button.description = "Pause"
            self.play_button.button_style = "danger"
            self.start_playback()
        else:
            self.play_button.description = "Play"
            self.play_button.button_style = "success"

    def start_playback(self):
        """
        Start playback in a loop until stopped or frames exhausted.
        """
        while self.playing and self.current_index < self.num_samples:
            self.display_frame(self.current_index)
            self.slider.value = self.current_index
            time.sleep(self.interval)  # Pause for the specified interval
            self.current_index += 1
            if self.current_index >= self.num_samples:
                self.playing = False
                self.play_button.value = False
                self.play_button.description = "Play"
                self.play_button.button_style = "success"

    def update_frame(self, change):
        """
        Update the displayed frame when the slider is adjusted.
        """
        self.current_index = change["new"]
        self.display_frame(self.current_index)

    def next_frame(self, _):
        """
        Display the next frame.
        """
        if self.current_index < self.num_samples - 1:
            self.current_index += 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)

    def previous_frame(self, _):
        """
        Display the previous frame.
        """
        if self.current_index > 0:
            self.current_index -= 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)




## Working Welll


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

class VideoPlayer:
    def __init__(self, inputs, predictions, targets, sources, image_names, rgb_pred=False, interval=0.0):
        """
        Initialize the VideoPlayer for interactive visualization.
        """
        self.inputs = inputs.cpu().numpy()
        self.predictions = predictions
        self.targets = targets if isinstance(targets, np.ndarray) else targets.cpu().numpy()
        self.sources = sources
        self.image_names = image_names
        self.rgb_pred = rgb_pred
        self.num_samples = inputs.shape[0]
        self.interval = interval  # Playback speed in seconds

        # Normalize inputs, predictions, and targets once
        self.normalized_inputs = (self.inputs.transpose(0, 2, 3, 1) + 1) / 2  # CHW to HWC and scale to [0, 1]
        self.normalized_predictions = (
            self.predictions / 255.0 if self.predictions.max() > 1 else self.predictions
        )
        self.normalized_targets = (
            self.targets / 255.0 if self.targets.max() > 1 else self.targets
        )

        # State variables
        self.current_index = 0
        self.playing = False

        # Widgets
        self.slider = widgets.IntSlider(value=0, min=0, max=self.num_samples - 1, step=1, description="Frame:")
        self.play_button = widgets.ToggleButton(value=False, description="Play", button_style="success")
        self.next_button = widgets.Button(description="Next")
        self.prev_button = widgets.Button(description="Previous")
        self.output = widgets.Output()

        # Event bindings
        self.play_button.observe(self.toggle_play, names="value")
        self.slider.observe(self.update_frame, names="value")
        self.next_button.on_click(self.next_frame)
        self.prev_button.on_click(self.previous_frame)

        # Layout
        self.controls = widgets.HBox([self.prev_button, self.play_button, self.next_button, self.slider])
        display(self.controls, self.output)

    def display_frame(self, index):
        """
        Display the frame corresponding to the given index.
        """
        with self.output:
            # Clear the output widget
            clear_output(wait=True)

            # Create a new figure for the current frame
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Input image
            axes[0].imshow(self.normalized_inputs[index], vmin=0, vmax=1)
            axes[0].set_title(f"Input Image (Frame {index})", fontsize=12)
            axes[0].axis("off")

            # Prediction
            if self.rgb_pred:
                axes[1].imshow(self.normalized_predictions[index], vmin=0, vmax=1)
            else:
                axes[1].imshow(self.normalized_predictions[index], cmap="gray", vmin=0, vmax=1)
            axes[1].set_title(f"Prediction (Frame {index})", fontsize=12)
            axes[1].axis("off")

            # Ground truth
            axes[2].imshow(self.normalized_targets[index], vmin=0, vmax=1)
            axes[2].set_title(f"Ground Truth (Frame {index})", fontsize=12)
            axes[2].axis("off")

            # Add source and image name
            fig.suptitle(
                f"Source: {self.sources[index]} | Image Name: {self.image_names[index]}",
                fontsize=14,
                y=0.98,
            )

            # Adjust layout and show
            plt.tight_layout()
            plt.show()

    def toggle_play(self, change):
        """
        Toggle play/pause functionality.
        """
        self.playing = change["new"]
        if self.playing:
            self.play_button.description = "Pause"
            self.play_button.button_style = "danger"
            self.start_playback()
        else:
            self.play_button.description = "Play"
            self.play_button.button_style = "success"

    def start_playback(self):
        """
        Start playback in a loop until stopped or frames exhausted.
        """
        while self.playing and self.current_index < self.num_samples:
            self.display_frame(self.current_index)
            self.slider.value = self.current_index
            time.sleep(self.interval)  # Pause for the specified interval
            self.current_index += 1
            if self.current_index >= self.num_samples:
                self.playing = False
                self.play_button.value = False
                self.play_button.description = "Play"
                self.play_button.button_style = "success"

    def update_frame(self, change):
        """
        Update the displayed frame when the slider is adjusted.
        """
        self.current_index = change["new"]
        self.display_frame(self.current_index)

    def next_frame(self, _):
        """
        Display the next frame.
        """
        if self.current_index < self.num_samples - 1:
            self.current_index += 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)

    def previous_frame(self, _):
        """
        Display the previous frame.
        """
        if self.current_index > 0:
            self.current_index -= 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)


# Example Usage
# Assuming inputs, predictions, and targets are already computed and available
# Replace `map_classes_to_colors` and other functions with your own utilities

for batch_idx, (inputs, targets, sources, image_names) in enumerate(val_loader):
    with torch.no_grad():
        inputs, targets = inputs.to(device), targets.to(device)
        predictions = model(inputs)
        predictions = torch.argmax(predictions, dim=1)  # Shape: [N, H, W]

        # Convert predictions and targets to RGB
        rgb_predictions = np.stack([
            map_classes_to_colors(predictions[i].cpu().numpy(), class_to_color) for i in range(predictions.shape[0])
        ])  # Shape: [N, H, W, 3]

        rgb_targets = np.stack([
            map_classes_to_colors(targets[i].cpu().numpy(), class_to_color) for i in range(targets.shape[0])
        ])

    # Initialize VideoPlayer with sources and image names
    VideoPlayer(inputs, rgb_predictions, rgb_targets, sources, image_names, rgb_pred=True)
    break

Output()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output
import time


class VideoPlayer:
    def __init__(self, inputs, predictions, targets, sources, image_names, rgb_pred=False, interval=0.0):
        """
        Initialize the VideoPlayer for interactive visualization.
        """
        # Precompute normalized data
        self.normalized_inputs = (inputs.cpu().numpy().transpose(0, 2, 3, 1) + 1) / 2  # CHW -> HWC, scale to [0, 1]
        self.normalized_predictions = (
            predictions / 255.0 if predictions.max() > 1 else predictions
        )
        self.normalized_targets = (
            targets / 255.0 if targets.max() > 1 else targets
        )

        # Store metadata
        self.sources = sources
        self.image_names = image_names
        self.rgb_pred = rgb_pred
        self.num_samples = inputs.shape[0]
        self.interval = interval  # Playback speed in seconds

        # State variables
        self.current_index = 0
        self.playing = False

        # Widgets
        self.slider = widgets.IntSlider(value=0, min=0, max=self.num_samples - 1, step=1, description="Frame:")
        self.play_button = widgets.ToggleButton(value=False, description="Play", button_style="success")
        self.next_button = widgets.Button(description="Next")
        self.prev_button = widgets.Button(description="Previous")
        self.output = widgets.Output()

        # Event bindings
        self.play_button.observe(self.toggle_play, names="value")
        self.slider.observe(self.update_frame, names="value")
        self.next_button.on_click(self.next_frame)
        self.prev_button.on_click(self.previous_frame)

        self.normalized_inputs = np.clip((inputs.cpu().numpy().transpose(0, 2, 3, 1) + 1) / 2, 0, 1)
        self.normalized_predictions = np.clip(predictions / 255.0, 0, 1)  # Normalize to [0, 1]
        self.normalized_targets = np.clip(targets / 255.0, 0, 1)          # Normalize to [0, 1]

        # Layout
        self.controls = widgets.HBox([self.prev_button, self.play_button, self.next_button, self.slider])
        display(self.controls, self.output)




    def display_frame(self, index):
        """
        Efficiently update the frame using persistent AxesImage objects.
        """
        with self.output:
            # Clear output widget
            clear_output(wait=True)

            if not hasattr(self, "fig") or self.fig is None:
                print("Creating figure for the first time")
                # Create figure and axes only once
                self.fig, self.axes = plt.subplots(1, 3, figsize=(15, 5))
                
                # Initialize AxesImage objects for updating
                self.input_im = self.axes[0].imshow(self.normalized_inputs[index], vmin=0, vmax=1)
                self.pred_im = self.axes[1].imshow(self.normalized_predictions[index], vmin=0, vmax=1)
                self.gt_im = self.axes[2].imshow(self.normalized_targets[index], vmin=0, vmax=1)

                # Set titles and axes properties
                self.axes[0].set_title("Input Image", fontsize=12)
                self.axes[0].axis("off")
                self.axes[1].set_title("Prediction", fontsize=12)
                self.axes[1].axis("off")
                self.axes[2].set_title("Ground Truth", fontsize=12)
                self.axes[2].axis("off")

                # Add metadata
                self.fig.suptitle(
                    f"Source: {self.sources[index]} | Image Name: {self.image_names[index]}",
                    fontsize=14,
                    y=0.98,
                )

                # Adjust layout
                plt.tight_layout()
                plt.show()
            else:
                print(f"Updating frame {index}")
                # Update existing AxesImage objects
                self.input_im.set_array(self.normalized_inputs[index])
                self.pred_im.set_array(self.normalized_predictions[index])
                self.gt_im.set_array(self.normalized_targets[index])

                # Update metadata and redraw
                self.fig.suptitle(
                    f"Source: {self.sources[index]} | Image Name: {self.image_names[index]}",
                    fontsize=14,
                    y=0.98,
                )
                self.fig.canvas.draw()
                self.fig.canvas.flush_events()  # Ensure immediate update

    def toggle_play(self, change):
        """
        Toggle play/pause functionality.
        """
        self.playing = change["new"]
        if self.playing:
            self.play_button.description = "Pause"
            self.play_button.button_style = "danger"
            self.start_playback()
        else:
            self.play_button.description = "Play"
            self.play_button.button_style = "success"

    def start_playback(self):
        """
        Start playback in a loop until stopped or frames exhausted.
        """
        while self.playing and self.current_index < self.num_samples:
            self.display_frame(self.current_index)
            self.slider.value = self.current_index
            time.sleep(self.interval)  # Pause for the specified interval
            self.current_index += 1
            if self.current_index >= self.num_samples:
                self.playing = False
                self.play_button.value = False
                self.play_button.description = "Play"
                self.play_button.button_style = "success"

    def update_frame(self, change):
        """
        Update the displayed frame when the slider is adjusted.
        """
        self.current_index = change["new"]
        self.display_frame(self.current_index)

    def next_frame(self, _):
        """
        Display the next frame.
        """
        if self.current_index < self.num_samples - 1:
            self.current_index += 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)

    def previous_frame(self, _):
        """
        Display the previous frame.
        """
        if self.current_index > 0:
            self.current_index -= 1
            self.slider.value = self.current_index
            self.display_frame(self.current_index)


# Example Usage
# Assuming inputs, predictions, and targets are already computed and available
# Replace `map_classes_to_colors` and other functions with your own utilities

for batch_idx, (inputs, targets, sources, image_names) in enumerate(val_loader):
    with torch.no_grad():
        inputs, targets = inputs.to(device), targets.to(device)
        predictions = model(inputs)
        predictions = torch.argmax(predictions, dim=1)  # Shape: [N, H, W]

        # Convert predictions and targets to RGB
        rgb_predictions = np.stack([
            map_classes_to_colors(predictions[i].cpu().numpy(), class_to_color) for i in range(predictions.shape[0])
        ])  # Shape: [N, H, W, 3]

        rgb_targets = np.stack([
            map_classes_to_colors(targets[i].cpu().numpy(), class_to_color) for i in range(targets.shape[0])
        ])
    print("it is starting")
    VideoPlayer(inputs, rgb_predictions, rgb_targets, sources, image_names, rgb_pred=True)
    # Initialize VideoPlayer with sources and image names
    break


it is starting


Output()